# HW 7:
This homework we are going to start with a dataset ingest, some cleaning and some visualizations. Then move to Streamlit

## US Census Data

Load the data posted on the github repo "us-population-2010-2019.csv"

In [1]:
import pandas as pd

df = pd.read_csv("us-population-2010-2019-states-code.csv")
df.head()

,states,states_code,id,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,Alabama,AL,1,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185"
1,Alaska,AK,2,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545"
2,Arizona,AZ,4,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717"
3,Arkansas,AR,5,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804"
4,California,CA,6,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223"


Using a lambda function, add a column to the dataframe that provides standard two letter abbreviations to all of the US States. For example, Connecticut would be CT.
https://www.50states.com/abbreviations.htm
*hint* there is a python package called "us"

In [2]:
# !pip install us
import us

# Package: https://pypi.org/project/us/
df['state_abbrv'] = df['states'].apply(lambda x: us.states.lookup(x).abbr if us.states.lookup(x) else None)
df.head()

,states,states_code,id,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,state_abbrv
0,Alabama,AL,1,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185",AL
1,Alaska,AK,2,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545",AK
2,Arizona,AZ,4,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717",AZ
3,Arkansas,AR,5,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804",AR
4,California,CA,6,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223",CA


Reshape your data, and create a new df called df_reshaped so that:

1. Convert 'year' column values to integers
2. Convert 'states' to strings
3. Get rid of the commas in the population numbers, and convert them to integers
4. Check your df_reshaped 

In [3]:
# Reshape to long format
df_reshaped = pd.melt(df, id_vars=['states', 'states_code', 'state_abbrv', 'id'], 
                      var_name='year', value_name='population')

# Convert 'year' to int
df_reshaped['year'] = df_reshaped['year'].astype(int)
print(df_reshaped['year'].dtype)

# Convert 'states' to string
df_reshaped['states'] = df_reshaped['states'].astype('string')
print(df_reshaped['states'].dtype)

# Remove commas in 'population', convert to int
df_reshaped['population'] = df_reshaped['population'].str.replace(',', '').astype(int)
print(df_reshaped['population'].dtype)

# Display first few rows
df_reshaped.head()

int64
string
int64


,states,states_code,state_abbrv,id,year,population
0,Alabama,AL,AL,1,2010,4785437
1,Alaska,AK,AK,2,2010,713910
2,Arizona,AZ,AZ,4,2010,6407172
3,Arkansas,AR,AR,5,2010,2921964
4,California,CA,CA,6,2010,37319502


Save your df_reshaped to a csv file

In [4]:
df_reshaped.to_csv('us-population-2010-2019-states-code-cleaned.csv', index=False)  

Subset your dataframe by selected_year = 2019 year create a new dataframe called df_selected_year


In [5]:
selected_year = 2019
df_selected_year = df_reshaped[df_reshaped["year"] == selected_year]
df_selected_year.head()

,states,states_code,state_abbrv,id,year,population
468,Alabama,AL,AL,1,2019,4903185
469,Alaska,AK,AK,2,2019,731545
470,Arizona,AZ,AZ,4,2019,7278717
471,Arkansas,AR,AR,5,2019,3017804
472,California,CA,CA,6,2019,39512223


Sort df_selected_year by population, from highest to lowest, and create a new df called "df_selected_year_sorted"

In [6]:
df_selected_year_sorted = df_selected_year.sort_values(by='population', ascending=False)
df_selected_year_sorted.head()

,states,states_code,state_abbrv,id,year,population
472,California,CA,CA,6,2019,39512223
511,Texas,TX,TX,48,2019,28995881
477,Florida,FL,FL,12,2019,21477737
500,New York,NY,NY,36,2019,19453561
506,Pennsylvania,PA,PA,42,2019,12801989


In [7]:
# Here is a function to calculate population difference between selected and previous year
def calculate_population_difference(input_df, input_year):
    # Filter data for the selected year and reset index
    selected_year_data = input_df[input_df['year'] == input_year].reset_index()
    # Filter data for the previous year and reset index
    previous_year_data = input_df[input_df['year'] == input_year - 1].reset_index()
    
    # Subtract population of the previous year from the selected year, assuming same row order
    selected_year_data['population_difference'] = selected_year_data.population.sub(previous_year_data.population, 
                                                                                    fill_value=0)
    # Calculate absolute value of the population difference
    selected_year_data['population_difference_absolute'] = abs(selected_year_data['population_difference'])
    
    # Return selected columns and sort by population difference
    return pd.concat([selected_year_data.states, selected_year_data.id, selected_year_data.population, 
                      selected_year_data.population_difference, selected_year_data.population_difference_absolute], 
                     axis=1).sort_values(by="population_difference", ascending=False)

In [8]:
df_population_difference_sorted = calculate_population_difference(df_reshaped, selected_year)
df_population_difference_sorted.head()

,states,id,population,population_difference,population_difference_absolute
43,Texas,48,28995881,367215,367215
9,Florida,12,21477737,233420,233420
2,Arizona,4,7278717,120693,120693
33,North Carolina,37,10488084,106469,106469
10,Georgia,13,10617423,106292,106292


In [9]:
# Filter states with population difference > 50000
df_greater_50000 = df_population_difference_sorted[df_population_difference_sorted["population_difference"] > 50000]
df_greater_50000

,states,id,population,population_difference,population_difference_absolute
43,Texas,48,28995881,367215,367215
9,Florida,12,21477737,233420,233420
2,Arizona,4,7278717,120693,120693
33,North Carolina,37,10488084,106469,106469
10,Georgia,13,10617423,106292,106292
47,Washington,53,7614893,91024,91024
5,Colorado,8,5758736,67449,67449
40,South Carolina,45,5148714,64558,64558
42,Tennessee,47,6829174,57543,57543
28,Nevada,32,3080156,52815,52815


In [10]:
# Calculate the % of States with population difference > 50000
percentage_change = "{:.2f}".format((df_greater_50000.shape[0]/df_population_difference_sorted.shape[0]) * 100)
print(f"% of States with Population Difference > 50000: {percentage_change} %")

% of States with Population Difference > 50000: 23.08 %


## Plots

### Heatmap: run the following code to see a heatmap

In [11]:
# !pip install altair
import altair as alt

alt.themes.enable("dark")

heatmap = alt.Chart(df_reshaped).mark_rect().encode(
        y=alt.Y('year:O', axis=alt.Axis(title="Year", titleFontSize=16, titlePadding=15, titleFontWeight=900, labelAngle=0)),
        x=alt.X('states:O', axis=alt.Axis(title="States", titleFontSize=16, titlePadding=15, titleFontWeight=900)),
        color=alt.Color('max(population):Q',
                         legend=alt.Legend(title=" "),
                         scale=alt.Scale(scheme="blueorange")),
        stroke=alt.value('black'),
        strokeWidth=alt.value(0.25),
        #tooltip=[
        #    alt.Tooltip('year:O', title='Year'),
        #    alt.Tooltip('population:Q', title='Population')
        #]
    ).properties(width=900
    #).configure_legend(orient='bottom', titleFontSize=16, labelFontSize=14, titlePadding=0
    #).configure_axisX(labelFontSize=14)
    ).configure_axis(
    labelFontSize=12,
    titleFontSize=12
    )

heatmap

alt.Chart(...)

### Choropleth: Run the following code to get a map of the population for the selected year above

In [12]:
# Choropleth via Altair
# !pip install vega_datasets
import altair as alt
from vega_datasets import data

alt.themes.enable("dark")

states = alt.topo_feature(data.us_10m.url, 'states')

alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('population:Q', scale=alt.Scale(scheme='blues')),   # scale=color_scale
    stroke=alt.value('#154360')
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df_selected_year, 'id', list(df_selected_year.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

alt.Chart(...)

Pycharm:
Create a pdf reader streamlit app in pycharm using the following code.Upload some pdfs and perform some queries. 
Below capture 5 queries and 5 responses. Check to make sure the responses are accurate. See if you can get your app to fail. Analyze why.

I read in one research paper here: https://www.tandfonline.com/doi/pdf/10.1080/00220388.2022.2120804#:~:text=We%20find%20that%20exposure%20to,difficulties%20in%20the%20last%2030days.

1. 
- Can you help me see what the journal that the paper is published in?
- The paper "Impact of Air Pollution on Mental Health in India" is published in The Journal of Development Studies.

2. 
- For pollution, on what kind of pollutant does this paper focus?
- The paper "Impact of Air Pollution on Mental Health in India" focuses on the impact of PM2.5 (particulate matter smaller than 2.5 micrometers) as a pollutant on mental health in India.

3. 
-  What empirical strategy does the paper use, and why can it demonstrate a causal relationship? Please explain how this instrument works.
-  The instrumental variable (IV) estimation strategy used in the paper helps demonstrate a causal relationship between air pollution exposure and mental health by addressing concerns such as omitted variable bias and reverse causality. The IV strategy uses the annual number of nighttime inversions as an instrument for air pollution exposure. Nighttime inversions are correlated with temperature, and since temperature can independently impact mental health outcomes, controlling for weather variables ensures the validity of the IV estimates. Additionally, the IV strategy helps ensure that individuals cannot predict their air pollution exposure based on inversion episodes, making the instrumental variable a reliable tool for establishing causality between air pollution exposure and mental health outcomes.

4. 
- Could you provide me with other papers that also utilize thermal inversion as an instrument and include the paper titles (not the citations)?
- Two papers that also utilize thermal inversion as an instrument for air pollution exposure are: Jans, Johansson, & Nilsson (2018) and Molina (2021).

5. 
- Could you provide me with a list of weather controls along with the units used by the authors in the regressions?
- The weather controls included in the regressions use units such as Celsius for temperature, centimeters per month for precipitation, meters per second for wind speed, and percentage for relative humidity.

In this question, I tried to input five questions to let the PDF reader Streamlit app help me analyze one economics research paper. For the first question, I want the app to directly take information from the header of the PDF. For the second question, I want the app to help me see the independent variable the paper is using, which entails searching for information. The third question aims to summarize the empirical strategy. The fourth one is about finding papers and seeing if the app can succeed in parsing the in-text citation and retrieving the paper title from the reference. The fifth one is to see if the app can read the tables. The app works well in all the questions. One exception is the fourth one, in which the app correctly identified the paper but did not return the titles of the relevant papers. This reflects the logistics of this program. It is good at searching for information from text chunks vectorizations with OpenAI embeddings and FAISS but lacks the capability to parse in-text citations and retrieve the corresponding paper titles from the reference.